# Utilisation de l'API GitHub

In [20]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

## On crawl le top 256 contributors 

In [183]:
token = "c91d944ae74a147fabd46e32be4b20e5a2647a3c"
headers = {'Authorization': 'token {}'.format(token)}

def get_top_users():
    req = requests.get("https://gist.github.com/paulmillr/2657075")
    if req.status_code == 200:
        soup = BeautifulSoup(req.content, "html.parser")
        table = soup.find("table").findAll("tr")[1:]
        return [row.find("td").text.split()[0] for row in table]
    else:
        print("Status Code Error")
        
def star_count(page):
    #for a given page, return the number of stars, and the number of repos
    stars = [(repos["name"],repos["stargazers_count"]) for repos in page]
    return stars

def all_page_count(user):
    i=1
    req = requests.get(f"https://api.github.com/users/{user}/repos?page={i}",headers=headers)
    page = req.json()
    all_repos = []
    while len(page) != 0:
        all_repos+=star_count(page)
        i+=1
        req= requests.get(f"https://api.github.com/users/{user}/repos?page={i}",headers=headers)
        page = req.json()
    return dict(all_repos)

def stars_users(list_user):
    #Creation d'un dictionnaire résumant le nombre de stars et de repos pour chaque user
    
    stars = {}
    stars["user"]=[]
    stars["#_repos"]=[]
    stars["tot_stars"]=[]
    T = len(list_user)
    k = 0
    for user in list_user:
        repos_user = all_page_count(user)
        stars["user"].append(user)
        stars["#_repos"].append(len(repos_user.keys()))
        stars["tot_stars"].append(sum(repos_user.values()))
        k+=1
        
        if k%10==0:
            print(f"Avancement : {round(k/T*100,2)}%")
    
    df=pd.DataFrame(stars)
    df["moyenne"]=round(df["tot_stars"]/df["#_repos"],2)
    return df.sort_values("moyenne", ascending=False)


In [184]:
list_users=get_top_users()

stars_users(list_users)

Avancement : 3.91%
Avancement : 7.81%
Avancement : 11.72%
Avancement : 15.62%
Avancement : 19.53%
Avancement : 23.44%
Avancement : 27.34%
Avancement : 31.25%
Avancement : 35.16%
Avancement : 39.06%
Avancement : 42.97%
Avancement : 46.88%
Avancement : 50.78%
Avancement : 54.69%
Avancement : 58.59%
Avancement : 62.5%
Avancement : 66.41%
Avancement : 70.31%
Avancement : 74.22%
Avancement : 78.12%
Avancement : 82.03%
Avancement : 85.94%
Avancement : 89.84%
Avancement : 93.75%
Avancement : 97.66%


,user,#_repos,tot_stars,moyenne
225,bailicangdu,13,45320,3486.15
18,kamranahmedse,43,99449,2312.77
142,vinta,32,58351,1823.47
117,mrdoob,35,55787,1593.91
194,IanLunn,18,25192,1399.56
44,sebastianbergmann,60,82471,1374.52
230,junyanz,14,19130,1366.43
153,nolimits4web,14,18681,1334.36
195,ruanyf,53,65454,1234.98
109,JakeWharton,100,107133,1071.33
